# ログデータにおけるテキストデータ累積出現回数の算出

様々な活動を記録したログデータにおいて、あるパターンの文字列が何回連続して登場したかをカウントすることが必要になる場合があります。これは集計処理とは異なるため、コーディングは多少ややこしくなります。
ここでは例として、サッカーJリーグの歴代優勝チームのリストから、各年代毎に最多連勝のケースを求めます。

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd

まず、Jリーグの歴代優勝チームを記録したデータファイルを読み込みます。

In [2]:
df1 = pd.read_excel('サッカーJリーグ.xlsx',sheet_name='winner')
df1.head()

,年代,年度,年間優勝
0,1990年代,1993.0,ヴェルディ川崎
1,1990年代,1994.0,ヴェルディ川崎
2,1990年代,1995.0,横浜マリノス
3,1990年代,1996.0,鹿島アントラーズ
4,1990年代,1997.0,ジュビロ磐田


次に、年代ごとに年間優勝チームの連続累積出現回数を求めます。

データフレームのあるカラムに対して、連続値の個数をカウントするには、値の変化点にフラグを立てて、cumsumで階段状の累積和を求めます。
この累積和をグループ番号とすることで、ユニークなグループ化ができます。
あとはgroupbyのcumcount()で各グループの個数をカウントします。

In [3]:
# 年代をインデックスに指定する
df1.set_index('年代',inplace=True)
df1['win_cum'] = df1.groupby(level=0)['年間優勝'].transform(lambda y:y.groupby((y != y.shift()).cumsum()).cumcount()+1)
df1.head()

,年度,年間優勝,win_cum
年代,,,
1990年代,1993.0,ヴェルディ川崎,1
1990年代,1994.0,ヴェルディ川崎,2
1990年代,1995.0,横浜マリノス,1
1990年代,1996.0,鹿島アントラーズ,1
1990年代,1997.0,ジュビロ磐田,1


次に、年代毎に連続累積出現回数が最大の行を抽出します。
最大値タイが複数ケースある場合は複数行抽出されます。

In [4]:
df2 = df1.set_index('年度',append=True)
df3 = df2[df2.groupby(level=0)['win_cum'].apply(lambda x: x==x.max())]
df3

年間優勝  win_cum
年代     年度                        
1990年代 1994.0    ヴェルディ川崎        2
2000年代 2009.0   鹿島アントラーズ        3
2010年代 2013.0  サンフレッチェ広島        2
       2018.0   川崎フロンターレ        2
2020年代 2021.0   川崎フロンターレ        2